In [28]:
import pandas as pd
import numpy as np
df_m = pd.read_csv('Data/MarvelComicCaracters.csv', sep=';')

# Описание данных

- name — оригинальное имя персонажа вселенной Marvel
- align — принадлежность персонажа к доброй/злой/нейтральной стороне
- eye — описание глаз персонажа
- hair — описание волос персонажа
- sex — пол персонажа
- appearances — количество появлений персонажа
- first_appearance — год первого появления персонажа

In [ ]:
df_m.head(n=3)

# Задания

### Сколько пропущенных значений в переменной `appearances`? Выберите один верный ответ.

In [15]:
df_m['appearances'].isna().sum()

1096

### Укажите, верно ли следующее утверждение: наибольший межквартильный размах по переменной `appearances` будет для злых персонажей. Вам понадобятся данные по типу персонажа (переменная `align`), удалите пропущенные значения по столбцу `appearances` и посчитайте требуемую меру разброса.

In [69]:
def get_iqr(a):
    q1 = np.percentile(a, 25)
    q3 = np.percentile(a, 75) 
    return q3 - q1

df_m \
    .loc[df_m['appearances'].notna(), :] \
    .groupby('align')['appearances'].agg(get_iqr) \
    .sort_values(ascending=False)

align
Добрый персонаж         13.0
Нейтральный персонаж     8.0
Злой персонаж            5.0
Name: appearances, dtype: float64

### Сколько выбросов встречается по переменной `appearances`? Работайте с исходными данными, предварительно удалите пропущенные значения по столбцу `appearances`, используйте для нахождения выбросов межквартильный размах. Выберите один верный ответ.

In [72]:
df_m \
    .loc[df_m['appearances'].notna(), :] \
    

,name,align,eye,hair,sex,appearances,first_appearance
0,Spider-Man (Peter Parker),Добрый персонаж,Карие глаза,Каштановые волосы,Мужской персонаж,4043.0,1962.0
1,Captain America (Steven Rogers),Добрый персонаж,Голубые глаза,Белые волосы,Мужской персонаж,3360.0,1941.0
2,"Wolverine (James \""Logan\"" Howlett)",Нейтральный персонаж,Голубые глаза,Черные волосы,Мужской персонаж,3061.0,1974.0
3,"Iron Man (Anthony \""Tony\"" Stark)",Добрый персонаж,Голубые глаза,Черные волосы,Мужской персонаж,2961.0,1963.0
4,Thor (Thor Odinson),Добрый персонаж,Голубые глаза,Русые волосы,Мужской персонаж,2258.0,1950.0
...,...,...,...,...,...,...,...
15275,Visher-Rakk (Earth-616),Злой персонаж,NaN,NaN,NaN,1.0,NaN
15276,Weapon Red (Chinese Mutant) (Earth-616),Злой персонаж,Белые глаза,Черные волосы,Мужской персонаж,1.0,NaN
15277,Wolfgang von Roehm (Earth-616),Нейтральный персонаж,NaN,NaN,Мужской персонаж,1.0,NaN
15278,Xan (Earth-616),Злой персонаж,Желтые глаза,Черные волосы,Мужской персонаж,1.0,NaN
